# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [17]:
# Tried to collect all import statements in one place

import numpy as np
import problem_unittests as tests
import string
from string import punctuation
from collections import Counter
import helper
import problem_unittests as tests
from torch.utils.data import TensorDataset, DataLoader

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [19]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [20]:
import problem_unittests as tests
import string
from string import punctuation
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function

    ## Build a dictionary that maps words to integers
    # NOTE: We use the `Counter()` and `sorted()` to make sure that 
    # the integer index reflects how common the word is in the corpus
    counts = Counter(text)
#     print('TEXT:', text)
#     print('Vocavulary size:', len(counts))
#     print(counts)
    vocab = sorted(counts, key=counts.get, reverse=True)
#     vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)} <-- That "1" caused A LOT OF ISSUES!!!...
    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    int_to_vocab = {ii: word for word, ii in vocab_to_int.items()}
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [21]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punct_dict = {}
    
    punct_dict['.'] =  '||Period||'
    punct_dict[','] =  '||Comma||'
    punct_dict['"'] =  '||Quotation_Mark||'
    punct_dict[';'] =  '||Semicolon||'  
#     punct_dict[':'] =  '||Colon||'
    punct_dict['!'] =  '||Exclamation_Mark||'
    punct_dict['?'] =  '||Question_Mark||'
    punct_dict['('] =  '||Left_Parentheses||'
    punct_dict[')'] =  '||Right_Parentheses||'
    punct_dict['-'] =  '||Dash||'
    punct_dict['\n'] = '||Return||'
        
    return punct_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

No GPU found. Please use a GPU to train your neural network.


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [25]:
from torch.utils.data import TensorDataset, DataLoader

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    
    # Total number of full sequences
    n_seq = len(words) - sequence_length 
    # Initialize the `features` and `targets` arrays (Numpy)
    features = np.zeros((n_seq, sequence_length), dtype=np.int64)
    targets = np.zeros((n_seq,), dtype=np.int64)
    # TEST
    # print('Features Array Size:', features.shape)
    # print('Targets Array Size: ', targets.shape)
    
    # Fill the `features` and `targets` arrays with the actual data from the passed text
    for i_seq in range(n_seq):
        features[i_seq,:] = words[i_seq:i_seq+sequence_length]
        targets[i_seq] = words[i_seq+sequence_length]
    # Create a TensorDataset from the features` and `targets`
    data = TensorDataset(torch.from_numpy(features), torch.from_numpy(targets))
    # Create a DataLoader
    data_loader = DataLoader(data, shuffle=True, batch_size=batch_size)
    # return a dataloader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [26]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[42, 43, 44, 45, 46],
        [ 8,  9, 10, 11, 12],
        [38, 39, 40, 41, 42],
        [18, 19, 20, 21, 22],
        [15, 16, 17, 18, 19],
        [14, 15, 16, 17, 18],
        [24, 25, 26, 27, 28],
        [19, 20, 21, 22, 23],
        [ 6,  7,  8,  9, 10],
        [30, 31, 32, 33, 34]])

torch.Size([10])
tensor([47, 13, 43, 23, 20, 19, 29, 24, 11, 35])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [27]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.vocab_size = vocab_size 
        self.output_size = output_size 
        self.embedding_dim = embedding_dim 
        self.hidden_dim = hidden_dim 
        self.n_layers = n_layers 
        self.dropout = dropout        
        
        # define model layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, 
                            num_layers=n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        # Get the batch size
        batch_size = nn_input.size(0)
        
        # 1. Embedding layer
        # embed_out = self.embed()  
        if train_on_gpu:
            embed_out = self.embed(torch.cuda.LongTensor(nn_input.tolist()))
        else:
            embed_out = self.embed(torch.LongTensor(nn_input.tolist()))
        # 2. LSTM layer
        lstm_out, hidden = self.lstm(embed_out, hidden)
        # Make sure to stack the outputs of the lstm to pass the LSTM's output 
        # to the fully-connected layer
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        # Apply dropout between the LSTM and the fully-connected layer
        lstm_out = self.dropout(lstm_out)
        # 3. Fully-connected layer (output)
        out = self.fc(lstm_out)
        
        # return one batch of output word scores and the hidden state
        
        # reshape into (batch_size, seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size)

#         print(type(out))
#         print(out.size())

        # get the last word in each sequence
        out = out[:, -1]
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        weight = next(self.parameters()).data
        
        # initialize hidden state with zero weights, and move to GPU if available
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(), 
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(), 
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [13]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
#     print('INPUT TYPE:', type(inp))
#     print('TARGET TYPE:', type(target))
#     print('HIDDEN TYPE:', type(hidden[0]))
    
    # move data to GPU, if available
    train_on_gpu = torch.cuda.is_available()
    
    if (train_on_gpu):
        rnn.cuda()
        inp, target = inp.cuda(), target.cuda()
    
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in hidden])

    # zero accumulated gradients
    rnn.zero_grad()
    
    # Forward pass
    output, h = rnn.forward(inp, h)
    
    # Estimate the error (aka loss)
    # Reminder: Output dimensions are (batch_size, seq_length, output_size)
    
    loss = criterion(output.squeeze(), target)
    
    # perform backpropagation and optimization
    # Option 1
    loss.backward()
    # Option 2
#     loss.backward(retain_graph=True)
    
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    clip = 5
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), h

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [28]:
# Data params
# Sequence Length
sequence_length = 12  # of words in a sequence
# Batch Size
batch_size = 300

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [42]:
# test dataloader

data_iter = iter(train_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
# print(sample_y)

torch.Size([300, 12])
tensor([[    6, 15258,    20,  ...,     2,   412,     1],
        [    0,    13,    11,  ...,   948,    20,   234],
        [   52,   878,   118,  ...,     9, 11045,    47],
        ...,
        [   18,     3,     0,  ...,     1,  7824,     1],
        [  387,  2047,    15,  ...,     1,     0,     0],
        [ 1947,  1028,    12,  ...,     0,     0,  2556]])

torch.Size([300])


In [29]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [46]:
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
print(rnn)

RNN(
  (embed): Embedding(21388, 300)
  (lstm): LSTM(300, 512, num_layers=3, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=21388, bias=True)
  (dropout): Dropout(p=0.3)
)


In [56]:
# from torchsummary import summary
# summary(rnn, input_size=(batch_size, sequence_length, hidden_dim)) #, (n_layers, batch_size, hidden_dim)))

# # TypeError: forward() missing 1 required positional argument: 'hidden'

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.656085224151611

Epoch:    1/10    Loss: 4.844678428649902

Epoch:    1/10    Loss: 4.602687619686127

Epoch:    1/10    Loss: 4.4793438148498534

Epoch:    1/10    Loss: 4.38421525478363

Epoch:    2/10    Loss: 4.254500853684553

Epoch:    2/10    Loss: 4.180574327945709

Epoch:    2/10    Loss: 4.145619118690491

Epoch:    2/10    Loss: 4.114975941181183

Epoch:    2/10    Loss: 4.104754632949829

Epoch:    3/10    Loss: 4.032561110568071

Epoch:    3/10    Loss: 3.9730004363059996

Epoch:    3/10    Loss: 3.938136867046356

Epoch:    3/10    Loss: 3.963045735836029

Epoch:    3/10    Loss: 3.9425267405509947

Epoch:    4/10    Loss: 3.864122591675369

Epoch:    4/10    Loss: 3.8262568793296814

Epoch:    4/10    Loss: 3.8275447144508363

Epoch:    4/10    Loss: 3.818289065361023

Epoch:    4/10    Loss: 3.8273019342422487

Epoch:    5/10    Loss: 3.7495168456070713

Epoch:    5/10    Loss: 3.7100558428764345

Epoch:    5/10    L

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


In [19]:
# Previous successful run
# NOTE: We used `sequence_length = 10`. All other hyperparameters were the same...
# -----------------------
"""
# Data params
# Sequence Length
sequence_length = 10  # of words in a sequence
# Batch Size
batch_size = 300

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500
"""

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.602660959243774

Epoch:    1/10    Loss: 4.842611843109131

Epoch:    1/10    Loss: 4.586328345298767

Epoch:    1/10    Loss: 4.456832783222199

Epoch:    1/10    Loss: 4.397507259368896

Epoch:    2/10    Loss: 4.268186329570115

Epoch:    2/10    Loss: 4.161361716747284

Epoch:    2/10    Loss: 4.150270639419555

Epoch:    2/10    Loss: 4.122827898979187

Epoch:    2/10    Loss: 4.089205041408539

Epoch:    3/10    Loss: 4.033663327745397

Epoch:    3/10    Loss: 3.9669179043769836

Epoch:    3/10    Loss: 3.9580869579315188

Epoch:    3/10    Loss: 3.938395736694336

Epoch:    3/10    Loss: 3.9342194075584414

Epoch:    4/10    Loss: 3.8694681425378845

Epoch:    4/10    Loss: 3.808638596534729

Epoch:    4/10    Loss: 3.8174706654548647

Epoch:    4/10    Loss: 3.814316719532013

Epoch:    4/10    Loss: 3.8067159276008606

Epoch:    5/10    Loss: 3.7647264307596457

Epoch:    5/10    Loss: 3.7103410248756408

Epoch:    5/10    

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?  

**Answer:**

 * Originally, there were a few general considerations that guided us in choosing specific values for the key hyperparameters:
     * Considering a fairly modest size of the vocabulary of the provided text corpus (21388), it made sense to use a lower value for the `embedding_dim` than in our previous RNN-related projects (e.g., "Word2Vec", "Sentiment Analysis with an RNN"). So, for the sake of reducing the computational cost we chose `embedding_dim = 200`.
     * Based on the ["Tips and Tricks" from Andrej Karpathy](https://github.com/karpathy/char-rnn#tips-and-tricks), we decided to use three LSTM layers in our network (`n_layers = 3`). It seemed like a good idea to be ambitious with the model complexity from the very beginning, even at the risk of overfitting: at least we would have an opportunity to find out early on "how much is too much". 
     * We also used a fairly "agressive" value for the learning rate (`learning_rate = 0.01`); we hoped that it would expedite the learing process, at least at the beginning. 
     * `sequence_length = 12`: Using 12 words to predict the next one appeared to be a good starting point (The choice of this parameter is a result of a trade-off: Choose too few words, and you risk to loose some of the context; choose too many, and you drive the computational cost up, plus you increase the possibility of adding some irrelevant, "out-of-scope" words).
     * `batch_size = 300`: This value was chosen to be pretty high for the sake of expediting the training process.
     
     
 * Observing the behavior of the loss during the training process provided us with some clues as to what values for the key hyperparameters would work better. Here is what we found after several attempts to train the model:
 

 1. At first, we tried the following values that determine the "complexity" of our neural network (see the comments above)
 :
     * `embedding_dim = 200`
     * `hidden_dim = 256`
     * `n_layers = 3`
  
  However, after training the model for just two epochs, we saw that the training loss had reached a saturation point: the loss curve got virtually flat. That behavior was a clear sign that our model was underfitting, i.e., it failed to capture and recognize more complex patterns in the data, which in turn drastically limited its performance. 
  
  
  2. Thus, we were forced to update the hyperparameters as follows:
     * `embedding_dim = 300`
     * `hidden_dim = 512`
     * `n_layers = 3`
     
     However, we discovered that having a high learning rate (0.01) leads to a "wrong" dynamics of the training loss: it was actually climbing up instead of going down as we would normally expect.
     
     
  3. After reducing the learning rate to 0.001, the overall behavior of the loss during the training process became much more acceptable: 
     * The train loss was gradually going down at a fairly decent pace. This suggested that the model was not underfitting anymore.
     * The loss curve didn't show any violent oscillations. This was a sign that the learning rate had been chosen appropriately.



 * __Outcome:__ With the network design and the hyperparameters described above, we were able to reduce the loss down to the recommended 3.5 by the 8th epoch. By the end of the training (10 epochs), the training loss was as low as 3.42 and thus, we passed another major requirement in the Project's rubric. The entire training run took just under 2 hours of GPU time in the project Workspace. 

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [1]:
! ls -lh

total 270M
drwxr-xr-x 2 root root 4.0K Apr  2 14:00 data
-rw-r--r-- 1 root root  66K Apr  4 20:02 dlnd_tv_script_generation.ipynb
-rw-r--r-- 1 root root  33K Mar  4 09:52 dlnd_tv_script_generation-zh.ipynb
-rw-r--r-- 1 root root 1.4K Apr  4 17:58 generated_script_1.txt
-rw-r--r-- 1 root root 2.6K Apr  4 18:00 generated_script_2.txt
-rw-r--r-- 1 root root 4.0K Apr  4 18:00 generated_script_3.txt
-rw-r--r-- 1 root root 1.4K Oct 16 00:27 helper.py
-rw-r--r-- 1 root root 2.5M Apr  4 16:05 preprocess.p
-rw-r--r-- 1 root root 9.4K Nov  6 23:42 problem_unittests.py
drwxr-xr-x 2 root root 4.0K Apr  3 18:32 __pycache__
-rw-r--r-- 1 root root  89M Apr  4 14:13 trained_rnn_cpu.pt
-rw-r--r-- 1 root root  89M Apr  4 17:56 trained_rnn.pt
-rw-r--r-- 1 root root  89M Apr  3 22:04 trained_rnn_v1.pt


In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

# Alternative Checkpoint (CPU)

**NOTE:** Since we had multiple CUDA-related issues with the trained model that had prevented us from using it for the script-generating task, let's try to do the following:
 1. While running in the GPU-enabled mode, load our trained model
 2. Move that model to CPU
 3. Save it in a separate file
 4. Switch to the GPU-disabled mode.
 5. Load the CPU version of the model

#### Step I: Run this in the GPU-enabled mode

In [12]:
# Move the trained model to CPU
trained_rnn = trained_rnn.to(torch.device("cpu"))
# saving the trained model on CPU
helper.save_model('./save/trained_rnn_cpu', trained_rnn)
print('The CPU version of the Model has been saved...')

The CPU version of the Model has been saved...


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#### Step II: Run this in the GPU-disabled mode

In [6]:
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn_cpu')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [26]:
# run the cell multiple times to get different results!
gen_length = 1200 # modify the length to your preference
prime_word = 'kramer' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], 
                            int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


kramer: importante windbreaker windbreaker windbreaker windbreaker windbreaker windbreaker microwave.

elaine: what? you know, i think it's not that.

jerry: i mean, you know, i don't know if i can go in my office.

jerry:(to elaine) i don't want to get a good job to make it.

elaine: well...(he gets up and starts looking in the couch) oh, hi.

jerry: oh, yeah, yeah.

jerry: i mean, i can't get a new one in the city...

george: i don't know. i don't know. i know.

elaine: i know. i was in the mood.

jerry: i can't believe i have a job.

jerry: what about?

kramer: oh..(to jerry) hey.

george: oh, you know, i don't know if you have the idea.

newman: i can't do.

elaine:(to elaine) oh, hi.

elaine: what are you doin'?

george: i don't know.

kramer: i don't want the money.

jerry: no, no, i can't.

kramer: oh, you know, i can't believe that.(they leave)

elaine:(to george) hey, you got to go?

elaine: i don't think so.

jerry: oh, i didn't think you should do it, you know, you know what

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [22]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

In [24]:
# save script to a text file
f =  open("generated_script_2.txt","w")
f.write(generated_script)
f.close()

In [27]:
# save script to a text file
f =  open("generated_script_3.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.

# Offical Review from Udacity

## Meets Specifications

> Excellent work with the project! :smile: 👏
> 
> I hope the review helped you. If you feel there's something more that you would have preferred from this review please leave a comment. That would immensely help me to improve feedback for any future reviews I conduct including for further projects. Would appreciate your input too. Thanks!
> 
> Congratulations on finishing the project! :smile: :udacious:

## All Required Files and Tests
### The project submission contains the project notebook, called “dlnd_tv_script_generation.ipynb”.

### All the unit tests in project have passed.

## Pre-processing Data
### The function create_lookup_tables create two dictionaries:

 * Dictionary to go from the words to an id, we'll call vocab_to_int
 * Dictionary to go from the id to word, we'll call int_to_vocab
The function create_lookup_tables return these dictionaries as a tuple (vocab_to_int, int_to_vocab).

> Good work! You correctly established the two dictionaries.
> 
> Also, here's a good resource if you wish to understand word embeddings a bit more as well - http://veredshwartz.blogspot.in/2016/01/representing-words.html

### The function token_lookup returns a dict that can correctly tokenizes the provided symbols.

> Good job.
> 
> Here's a good resource discussing more preprocessing steps that you can try - http://datascience.stackexchange.com/questions/11402/preprocessing-text-before-use-rnn

## Batching Data
### The function batch_data breaks up word id's into the appropriate sequence lengths, such that only complete sequence lengths are constructed.

> Nice work!
> 
> You utilized a for loop to create your array and fill the indices with the required values.
> 
> Do you think you could complete this step without using a for loop and only using numpy? Numpy is optimized to handle complex operations so that you don't usually require loops and is faster as a result. So, try it out and see if you can optimize your code or not :)

### In the function batch_data, data is converted into Tensors and formatted with TensorDataset.

> Great job!
> 
> For your own benefit, you could also try to create your own function which utilizes Python Generators. If you haven't worked with Generators before, I highly recommend you try them out for batching the data instead of using TensorDataset and DataLoader. Generators probably don't offer any "distinct" advantages over the two, but it's a good python concept to be familiar with as per me.
> 
> Here's a couple of resources on generators that I found useful when I was trying to learn them -
> 
 * https://www.youtube.com/watch?v=bD05uGo_sVI
 * https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/

### Finally, batch_data returns a DataLoader for the batched training data.

## Build the RNN
### The RNN class has complete __init__, forward , and init_hidden functions.

> Well done!
> 
> Excellent work!
> 
> I have some questions for you to think about that might benefit you -
> 
> Does your nn.Linear() layer require any activation function?
> If you wanted to expand your class to be more generic such that you could have any number of layers you wanted, how would you go about that? Is that required (not just for this project, but for a general implementation which you could use anywhere for example)
> Are you familiar with Batch Normalization, yet? It's alright if not as I think you will come across this in the Classroom soon. But look it up and see how it compares to dropouts and whether they are useful for RNNs.
> The reason for the above questions is to try to make sure you are ready to explore alternatives. Often, Udacity's helper code tends to tie us down a bit and we restrict ourselves from exploring further. So try to think about the above and get the answers even if they don't seem valid for this project! :)

### The RNN must include an LSTM or GRU and at least one fully-connected layer. The LSTM/GRU should be correctly initialized, where relevant.

> Nicely done!
> 
> Especially for using the Embedding Layer. Have you checked out the documentation for it? Check it out - https://pytorch.org/docs/stable/nn.html#embedding and try to play around with some of the additional parameters for this method.
> 
> Also, here's a good resource if you wish to understand word embeddings a bit as well - http://veredshwartz.blogspot.in/2016/01/representing-words.html

## RNN Training
 * Enough epochs to get near a minimum in the training loss, no real upper limit on this. Just need to make sure the training loss is low and not improving much with more training.
 * Batch size is large enough to train efficiently, but small enough to fit the data in memory. No real “best” value here, depends on GPU memory usually.
 * Embedding dimension, significantly smaller than the size of the vocabulary, if you choose to use word embeddings
 * Hidden dimension (number of units in the hidden layers of the RNN) is large enough to fit the data well. Again, no real “best” value.
 * n_layers (number of layers in a GRU/LSTM) is between 1-3.
 * The sequence length (seq_length) here should be about the size of the length of sentences you want to look at before you generate the next word.
 * The learning rate shouldn’t be too large because the training algorithm won’t converge. But needs to be large enough that training doesn’t take forever.
> You selected a good set of hyperparameters.
> 
> * You get a low training loss with the number of epochs, which is good. We should tend to select the number of epochs such that you get a low training loss which reaches kind of a steady-state (not much change in value beyond a point).
> * I would recommend that you use a smaller batch size. Try to observe what happens if you lower it or increase it, first. Smaller batch sizes take too long to train. Larger batch sizes speed up the training but can degrade the quality of the model. Here is a useful resource to understand this better - http://stats.stackexchange.com/questions/164876/tradeoff-batch-size-vs-number-of-iterations-to-train-a-neural-network Also, do you think the batch size should be a power of 2 (like 16, 32, 64, etc) or your value of 300 is fine? Do you think these DL frameworks and tools are optimized to use values that are powers of 2 because of the hardware?
> * Your current RNN size (number of layers) fits the data well. What results do you think you will get when you increase or decrease that? First try increasing and see if your model converges better and has a lower loss or not. Changing this is limited to your system configuration as well, so you might not be able to have a very high value either.
> * Usually, we select the sequence length so that it matches the structure of data we are working with. There isn't really a rule of thumb though -http://stats.stackexchange.com/questions/158834/what-is-a-feasible-sequence-length-for-an-rnn-to-model. For a tougher challenge, try to think of having variable sequence lengths. https://towardsdatascience.com/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e might help in this regard.
> * Good selection for the learning rate!
> 
> Couple of good things for you to try out - a slightly smaller learning rate with more epochs. As you can see your loss seems to still decrease, that means your model seems to still be learning so you can train for more epochs. But at the same time your learning rate results in the loss being a bit spiky (increases and decreases a bit). That usually means a higher learning rate. So you can try to reduce that as well.
> 
> Good work!

### The printed loss should decrease during training. The loss should reach a value lower than 3.5.

> Excellent job! You can still experiment more with your hyperparameters, if you'd like to. But you are getting good results.

### There is a provided answer that justifies choices about model size, sequence length, and other parameters.

> Well explained. You are starting to get an intuitive understanding of working with RNNs/LSTMs 🙂

> But I would recommend focussing less on expediting the training and more on understanding the fundamentals around hyperparameter tuning as well.

## Generate TV Script
### The generated script can vary in length, and should look structurally similar to the TV script in the dataset.

### It doesn’t have to be grammatically correct or make sense.

> Your generated script looks awesome! :clap:
> 
> Question for you to think about - How do you think you could design your model so that you can control how many words per dialogue your script has? Do you think that's possible with just deep learning?